In [1]:
# Install required packages
# !pip install dspy-ai litellm mlflow python-dotenv

import dspy
import mlflow
import os
from dotenv import load_dotenv
from typing import List

print("✅ All imports successful!")

✅ All imports successful!


In [2]:
# Load environment variables
load_dotenv()

# Set up MLflow experiment
mlflow.set_experiment("DSPy-Optimizer-Groq")

# Enable comprehensive tracing for optimization
mlflow.dspy.autolog(
    log_evals=True,           # Log evaluation results
    log_compiles=True,        # Log optimization process
    log_traces_from_compile=True  # Trace during optimization
)

print("🔍 MLflow tracing enabled with optimization tracking!")
print(f"📊 Experiment: DSPy-Optimizer-Groq")


2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/04 14:17:50 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/04 14:17:50 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/04 14:17:50 INFO mlflow.tracking.fluent: Experiment with name 'DSPy-Optimizer-Groq' does not exist. Creating a new experiment.


🔍 MLflow tracing enabled with optimization tracking!
📊 Experiment: DSPy-Optimizer-Groq


In [3]:
# Configure Groq with Llama 3.3 70B Versatile
lm = dspy.LM(
    'groq/llama-3.3-70b-versatile',
    api_key=os.getenv('GROQ_API_KEY'),
    max_tokens=1024,
    temperature=0.7
)

# Set as default LM
dspy.configure(lm=lm)

print("🚀 Groq (Llama 3.3 70B Versatile) configured!")
print(f"Model: {lm.model}")
print(f"📊 Free tier limits:")
print(f"   - Requests: 30/min, 1K/day")
print(f"   - Tokens: 12K/min, 100K/day")

# Quick test
test = dspy.Predict("question -> answer")
result = test(question="What is the capital of France?")
print(f"\nTest: Capital of France = {result.answer}")
print("✅ Groq connection verified!")


🚀 Groq (Llama 3.3 70B Versatile) configured!
Model: groq/llama-3.3-70b-versatile
📊 Free tier limits:
   - Requests: 30/min, 1K/day
   - Tokens: 12K/min, 100K/day

Test: Capital of France = The capital of France is Paris.
✅ Groq connection verified!


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## an...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(


In [4]:
# Mock Wikipedia database (since public ColBERT servers are often down)
WIKIPEDIA_MOCK = {
    "python programming": [
        "Python is a high-level programming language created by Guido van Rossum in 1991.",
        "Python emphasizes code readability with significant whitespace and supports multiple programming paradigms.",
        "Python is widely used in web development, data science, and machine learning."
    ],
    "machine learning": [
        "Machine learning is a subset of artificial intelligence that enables systems to learn from data.",
        "Common machine learning algorithms include decision trees, neural networks, and support vector machines.",
        "Machine learning is used in recommendation systems, image recognition, and natural language processing."
    ],
    "eiffel tower": [
        "The Eiffel Tower is located in Paris, France and was completed in 1889.",
        "The tower stands 330 meters tall and was designed by Gustave Eiffel.",
        "It was initially criticized but is now one of the most visited monuments in the world."
    ],
    "albert einstein": [
        "Albert Einstein was a theoretical physicist born in 1879 in Germany.",
        "He developed the theory of relativity, with E=mc² being his most famous equation.",
        "Einstein received the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect."
    ]
}

def search_wikipedia(query: str) -> List[str]:
    """
    Search Wikipedia for relevant passages.
    Returns top 3 most relevant text passages.
    
    Args:
        query: Search query string
    
    Returns:
        List of relevant text passages from Wikipedia
    """
    query_lower = query.lower()
    
    # Simple keyword matching
    results = []
    for topic, passages in WIKIPEDIA_MOCK.items():
        if any(word in query_lower for word in topic.split()):
            results.extend(passages)
    
    # If no matches, return general info
    if not results:
        results = ["No specific information found. Please try a more specific query."]
    
    return results[:3]  # Return top 3 results


print("📚 Mock Wikipedia search tool created")
print("\n🔍 Test search:")
test_results = search_wikipedia("What is Python?")
for i, result in enumerate(test_results, 1):
    print(f"  {i}. {result[:80]}...")


📚 Mock Wikipedia search tool created

🔍 Test search:
  1. Python is a high-level programming language created by Guido van Rossum in 1991....
  2. Python emphasizes code readability with significant whitespace and supports mult...
  3. Python is widely used in web development, data science, and machine learning....


In [5]:
# Create ReAct agent with Wikipedia search tool
react_agent = dspy.ReAct(
    signature="question -> answer",
    tools=[search_wikipedia],
    max_iters=5
)

print("🤖 ReAct RAG Agent created")
print("   - Signature: question -> answer")
print("   - Tools: search_wikipedia")
print("   - Max iterations: 5")


🤖 ReAct RAG Agent created
   - Signature: question -> answer
   - Tools: search_wikipedia
   - Max iterations: 5


In [6]:
# Create training dataset (small dataset is sufficient for DSPy)
trainset = [
    dspy.Example(
        question="What programming language was created by Guido van Rossum?",
        answer="Python"
    ).with_inputs("question"),
    
    dspy.Example(
        question="When was Python first released?",
        answer="1991"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What is a subset of AI that learns from data?",
        answer="Machine learning"
    ).with_inputs("question"),
    
    dspy.Example(
        question="Where is the Eiffel Tower located?",
        answer="Paris"
    ).with_inputs("question"),
    
    dspy.Example(
        question="Who designed the Eiffel Tower?",
        answer="Gustave Eiffel"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What year was Albert Einstein born?",
        answer="1879"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What is Einstein's famous equation?",
        answer="E=mc²"
    ).with_inputs("question"),
]

# Create validation dataset
valset = [
    dspy.Example(
        question="What does Python emphasize in code?",
        answer="readability"
    ).with_inputs("question"),
    
    dspy.Example(
        question="When was the Eiffel Tower completed?",
        answer="1889"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What prize did Einstein win in 1921?",
        answer="Nobel Prize in Physics"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What are common machine learning algorithms?",
        answer="decision trees"
    ).with_inputs("question"),
]

print(f"📊 Datasets created:")
print(f"   - Training set: {len(trainset)} examples")
print(f"   - Validation set: {len(valset)} examples")
print(f"\n📝 Sample training example:")
print(f"   Question: {trainset[0].question}")
print(f"   Answer: {trainset[0].answer}")


📊 Datasets created:
   - Training set: 7 examples
   - Validation set: 4 examples

📝 Sample training example:
   Question: What programming language was created by Guido van Rossum?
   Answer: Python


In [7]:
print("🧪 Testing baseline agent (before optimization)\n")

# Test on a few examples
test_questions = [
    "What programming language was created by Guido van Rossum?",
    "Where is the Eiffel Tower located?",
    "What is Einstein's famous equation?"
]

for question in test_questions:
    result = react_agent(question=question)
    print(f"Q: {question}")
    print(f"A: {result.answer}")
    print("-" * 60)


🧪 Testing baseline agent (before optimization)



d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=

Q: What programming language was created by Guido van Rossum?
A: Python
------------------------------------------------------------


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=

Q: Where is the Eiffel Tower located?
A: The Eiffel Tower is located in Paris, France.
------------------------------------------------------------


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=

Q: What is Einstein's famous equation?
A: E=mc^2
------------------------------------------------------------


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(


In [8]:
# Use built-in exact match metric with flexibility
def custom_metric(example, pred, trace=None):
    """
    Custom metric that checks if prediction matches answer.
    Allows partial matches for better optimization.
    """
    # Use DSPy's built-in answer matching with partial match (F1 > 0.5)
    return dspy.evaluate.answer_exact_match(
        example, 
        pred, 
        trace=trace,
        frac=0.5  # Allow 50% F1 match (more lenient than exact match)
    )

print("✅ Evaluation metric defined: custom_metric (F1 >= 0.5)")


✅ Evaluation metric defined: custom_metric (F1 >= 0.5)
